# Components

In [61]:
import os
from langchain_community.embeddings import GPT4AllEmbeddings
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS

embeddings = GPT4AllEmbeddings()
index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={}
)

In [62]:
def logging(filename: str, content: str):
    with open(f"{filename}.txt", "w", encoding="utf-8") as file:
        file.write(content)

In [63]:
from openai import OpenAI

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
model_name = "gpt-4o"

# client = OpenAI(
#     api_key=os.environ.get("LLAMA_API_KEY"),
#     base_url="https://api.llama-api.com/"
# )
# model_name = "llama3.2-3b"

def ask_gpt(prompt: str) -> str:
    response = client.chat.completions.create(
        model=model_name,
        messages=[
            {
                "role": "user",
                "content": prompt,
            }
        ],
        max_tokens=512
    )
    return response.choices[0].message.content

In [ ]:
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langgraph.graph import START, StateGraph
from typing_extensions import List, TypedDict
from langchain.document_loaders import PyPDFLoader

def chunking(sauce: str, chunk_size, chunk_overlap):
    print("Begin chunking...")
    loader = PyPDFLoader(sauce)
    docs = loader.load()
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    print("Done!")
    return text_splitter.split_documents(docs)

# Path to the saved vector store
faiss_index_path = "faiss_index"
if os.path.exists(faiss_index_path):
    vector_store = FAISS.load_local(faiss_index_path, embeddings, allow_dangerous_deserialization=True)
else:
    all_splits = chunking("trietly.pdf", 1800, 200)
    print("Begin vector storing...")
    _ = vector_store.add_documents(documents=all_splits)
    print("Done vector storing!")
    vector_store.save_local("faiss_index")

retriever = vector_store.as_retriever(search_kwargs={"k": 5})

In [65]:
prompt_res = '''
Đọc thông tin và các ví dụ sau:
________________
1. Thông tin
________________
{context}
________________
2. Ví dụ
- Văn bản: Môi trường là tổng thể các yếu tố tự nhiên và nhân tạo xung quanh con người, ảnh hưởng đến sự sống và phát triển của mọi sinh vật.
Câu hỏi: Môi trường là gì?
- Văn bản: Yêu thương con người là sự quan tâm, giúp đỡ người khác, làm những điều tốt đẹp cho người khác, nhất là những người khó khăn, hoạn nạn.
Câu hỏi: Thế nào là yêu thương con người?
- Văn bản: Chương trình Cặp lá yêu thương là dự án hỗ trợ các em học sinh có hoàn cảnh khó khăn, vươn lên trong cuộc sống để tiếp tục tới trường.
Câu hỏi: Tìm hiểu và cho biết một chương trình giải trí với ý nghĩa thể hiện tình yêu thương của con người với những hoàn cảnh khó khăn?
- Văn bản: Yêu thương con người là tình cảm quý giá, một giá trị nhân văn và là truyền thống quý báu của dân tộc mà mỗi chúng ta cần phải giữ gìn và phát huy.
Câu hỏi: Giá trị của tình yêu thương đối với con người và xã hội là gì?
- Văn bản: Tình yêu thương con người mang lại niềm vui, sự tin tưởng vào bản thân và cuộc sống. 
Câu hỏi: Yêu thương con người chúng ta sẽ nhận được điều gì?
________________
Hãy đặt khoảng 5 câu hỏi cho văn bản dưới đây.
{statement}
'''

In [66]:
# Define state for application
class State(TypedDict):
    statement: str
    context: List[Document]
    answer: str

# Define application steps
def retrieve(state: State):
    print("Begin retriving!")
    docs = retriever.invoke(state["statement"])
    print("Done retrieving!")
    return {"context": docs}

def generate(state: State):
    print("Begin generating!")
    docs_content = "\n\n".join(doc.page_content for doc in state["context"])
    messages = prompt_res.format(statement = state["statement"], context = docs_content)
    messages.replace("\t", " ")
    logging("Prompt", messages)
    response = ask_gpt(messages)
    print("Done generating!")
    return {"answer": response}

print("Compiling...")
graph_builder = StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START, "retrieve")
graph = graph_builder.compile()
print("Done compiling!")

Compiling...
Done compiling!


In [67]:
statement = '''
Tuổi trẻ là tài sản lớn nhất, phải quý trọng thời gian, đừng sợ nghèo khó, gian nan. 
Hãy dùng tuổi trẻ để bồi dưỡng bản thân, hiểu được cái gì là đáng quý, biết được thứ nên đầu tư, chỗ nên tiết kiệm. 
Đó chính là điểm mấu chốt của bài học làm giàu, chìa khóa để thay đổi cuộc đời.
'''

response = graph.invoke({"statement": statement})
logging("result", response["answer"])

Begin retriving!
Done retrieving!
Begin generating!
Done generating!
